# Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Reading the dataset
df = pd.read_excel("/content/Omdena Chatbot Dataset.xlsx")
df.head(6)

,Tag,Pattern,Response
0,Greeting,Hi,Hi stranger
1,NaN,Hi there,hi too
2,NaN,hey,hello
3,NaN,How is you?,hello there
4,NaN,afternoon,How can I help you?
5,NaN,hello,NaN


In [ ]:
#Filling the missing values
# for tag we fill with before value
df.fillna(method='ffill', inplace=True)
df.head()

,Tag,Pattern,Response
0,Greeting,Hi,Hi stranger
1,Greeting,Hi there,hi too
2,Greeting,hey,hello
3,Greeting,How is you?,hello there
4,Greeting,afternoon,How can I help you?


Cleaning the Pattern Column

In [ ]:
df.head()

,Tag,Pattern,Response
0,Greeting,Hi,Hi stranger
1,Greeting,Hi there,hi too
2,Greeting,hey,hello
3,Greeting,How is you?,hello there
4,Greeting,afternoon,How can I help you?


In [ ]:
#A Function for cleaning the file (The Pattern column in it)
def text_clean(df):
  #Lowercasing all the letters
  df['Pattern'] = df['Pattern'].str.lower() 


  #Removing punctuations and replacing with a single space
  df['Pattern'] = df['Pattern'].str.replace(r'[()!?]', ' ', regex=True)  
  df['Pattern'] = df['Pattern'].str.replace(r'\[.*?\]', ' ', regex=True)

  #Filtering non-alphanumeric characters
  df['Pattern'] = df['Pattern'].str.replace(r'[^a-z0-9]', ' ', regex=True) 

  #Removing Stoping words 
  stop = stopwords.words('english') 
  df['Pattern_without_stopwords'] = df['Pattern'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
df.head()

,Tag,Pattern,Response
0,Greeting,hi,Hi stranger
1,Greeting,hi there,hi too
2,Greeting,hey,hello
3,Greeting,how is you,hello there
4,Greeting,afternoon,How can I help you?


In [ ]:
text_clean(df)
# tokenize the tweets
df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row['Pattern_without_stopwords']), axis=1)
df.head(20)

,Tag,Pattern,Response,Pattern_without_stopwords,tokenized_sents
0,Greeting,hi,Hi stranger,hi,[hi]
1,Greeting,hi there,hi too,hi,[hi]
2,Greeting,hey,hello,hey,[hey]
3,Greeting,how is you,hello there,,[]
4,Greeting,afternoon,How can I help you?,afternoon,[afternoon]
5,Greeting,hello,How can I help you?,hello,[hello]
6,Greeting,hello there,How can I help you?,hello,[hello]
7,Greeting,evening,How can I help you?,evening,[evening]
8,Greeting,morning,How can I help you?,morning,[morning]
9,Greeting,morning,How can I help you?,morning,[morning]


In [ ]:
#Creating A file of the df_tweets
df.to_csv('df_clean.csv')